In [97]:
import os
import sys
import time
import serial
import math
import numpy as np

# Adding the src folder in the current directory as it contains the script
# with the Thymio class
sys.path.insert(0, os.path.join(os.getcwd(), 'utils'))
from Thymio import Thymio

class robot():
    def __init__(self, th):
        self.curr_pos = [0,0,0]
        self.speed = 100
        self.th = th
        
    def get_position(self):
        return self.curr_pos
        
    def move_to_target(self, target_pos):
        if target_pos == self.curr_pos[0:2]: return False #if the robot is already at the position or doesn't move
       
        #distance from current position to target
        distance = math.sqrt(((target_pos[0]-self.curr_pos[0])**2)+((target_pos[1]-self.curr_pos[1])**2))
        
        #absolute angle from current position to target (this angle will always be returned between ]-pi;pi])
        path_angle = math.degrees(math.atan2(target_pos[1]-self.curr_pos[1],target_pos[0]-self.curr_pos[0]))
        
        #turn angle to get to target relative to Thymio frame
        turn_angle = path_angle - self.curr_pos[2]
        if abs(turn_angle) > 180:
             turn_angle = (turn_angle + 360) % 360;
                
        #give commands
        self.turn(turn_angle)
        self.forward(distance)

        #update position and angle of the robot
        self.curr_pos = [target_pos[0],target_pos[1],path_angle]
        
    def turn(self, turn_angle):  #input must be in degrees
        print("turn_angle:{}".format(turn_angle),"\n")

        if turn_angle > 0: #turn_angle to the left (180 case will also turn left)
            target_time = abs(turn_angle)/38.558 #linear fit model from rad to s for v=100 (change to Kalman)
            left_speed = 2**16-self.speed 
            right_speed = self.speed
            
        elif turn_angle < 0: #turn_angle to the right
            target_time = abs(turn_angle)/38.558 
            left_speed = self.speed
            right_speed = 2**16-self.speed
            
        else: #if turn_angle = 0, do not waste time
            return False
            
        print("target_time_turn:{}".format(target_time))
        t_0 = time.time()
        
        self.th.set_var("motor.left.target", left_speed)
        self.th.set_var("motor.right.target", right_speed)
        
        time.sleep(target_time)
        t_end = time.time()
        print("actual_time_turn:{}".format(t_end-t_0))
        
        #time.sleep(0.1)

    def forward(self, distance): 
        print("distance:{}".format(distance))

        target_time = distance/31.573 #linear fit model from mm to s for v=100 (change to Kalman)
        
        print("target_time_forward:{}".format(target_time))
        t_0 = time.time()
        
        self.th.set_var("motor.left.target", self.speed)
        self.th.set_var("motor.right.target", self.speed)
        
        time.sleep(target_time)
        t_end = time.time()
        print("actual_time_forward:{}".format(t_end-t_0), "\n")
        
        #time.sleep(0.1)
        
    def stop(self):
        self.th.set_var("motor.left.target", 0)
        self.th.set_var("motor.right.target", 0)
        
        #time.sleep(0.1)


In [3]:
th = Thymio.serial(port="/dev/cu.usbmodem141401", refreshing_rate=0.1)

time.sleep(3) # To make sure the Thymio has had time to connect

variables = th.variable_description()

for var in variables : #If this does not print, reconnect USB!
    print(var)


{'name': '_id', 'offset': 0, 'size': 1}
{'name': 'event.source', 'offset': 1, 'size': 1}
{'name': 'event.args', 'offset': 2, 'size': 32}
{'name': '_fwversion', 'offset': 34, 'size': 2}
{'name': '_productId', 'offset': 36, 'size': 1}
{'name': 'buttons._raw', 'offset': 37, 'size': 5}
{'name': 'button.backward', 'offset': 42, 'size': 1}
{'name': 'button.left', 'offset': 43, 'size': 1}
{'name': 'button.center', 'offset': 44, 'size': 1}
{'name': 'button.forward', 'offset': 45, 'size': 1}
{'name': 'button.right', 'offset': 46, 'size': 1}
{'name': 'buttons._mean', 'offset': 47, 'size': 5}
{'name': 'buttons._noise', 'offset': 52, 'size': 5}
{'name': 'prox.horizontal', 'offset': 57, 'size': 7}
{'name': 'prox.comm.rx._payloads', 'offset': 64, 'size': 7}
{'name': 'prox.comm.rx._intensities', 'offset': 71, 'size': 7}
{'name': 'prox.comm.rx', 'offset': 78, 'size': 1}
{'name': 'prox.comm.tx', 'offset': 79, 'size': 1}
{'name': 'prox.ground.ambiant', 'offset': 80, 'size': 2}
{'name': 'prox.ground.refl

In [99]:
my_th = robot(th)

my_th.forward(10)
my_th.forward(100)
my_th.turn(180)
my_th.turn(360)
my_th.turn(-720)
my_th.move_to_target([0,1])
my_th.move_to_target([12,33])

full_path = np.array([[0,1,1,2,3,4,5,5,5,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,7,7,7,7,7,8,9,10,11,12,13,14,15,16,17,18,19,20],
 [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,38,39,39,39,40]])

full_path = full_path.T

for i in range(len(full_path)):
    my_th.move_to_target([full_path[i,0],full_path[i,1]])
    pos_th = my_th.get_position()
    print("Thymio position: {}".format(pos_th))
    
my_th.stop()
        

distance:10
target_time_forward:0.31672631678966207
actual_time_forward:0.3177361488342285 

distance:100
target_time_forward:3.1672631678966203
actual_time_forward:3.170325994491577 

turn_angle:180 

target_time_turn:4.668291923854972
actual_time_turn:4.672255992889404
turn_angle:360 

target_time_turn:9.336583847709944
actual_time_turn:9.338398933410645
turn_angle:-720 

target_time_turn:18.673167695419888
actual_time_turn:18.67963719367981
turn_angle:90.0 

target_time_turn:2.334145961927486
actual_time_turn:2.3403048515319824
distance:1.0
target_time_forward:0.0316726316789662
actual_time_forward:0.03330421447753906 

turn_angle:-20.556045219583467 

target_time_turn:0.5331201104721061
actual_time_turn:0.5371172428131104
distance:34.17601498127012
target_time_forward:1.0824443347565997
actual_time_forward:1.08579421043396 

turn_angle:-179.42706130231653 

target_time_turn:4.653432784436863
actual_time_turn:4.656741142272949
distance:35.11409973215888
target_time_forward:1.1121559

In [88]:
my_th.stop()

#     def measurement_model(self):
#         #distance
#         x_d = np.linspace(0.5, 5, 10)
#         y_d = [16,32.5,46.5,61,79.5,95,109,127.5,143,159.5]
        
#         diff_y_d = np.diff(y_d) # Calculating difference list 
#         mean_d = np.mean(diff_y_d) #15.94444
#         sigma_d = np.std(diff_y_d) #6.877318
#         distance_p0 = [1, mean_d, sigma_d]

#         #angle
#         x_a = np.linspace(0, 9, 10)
#         y_a = [0,42,78,114,150,194,231,270,310,347]

#         diff_y_a = np.diff(y_a) # Calculating difference list 
#         mean_a = np.mean(diff_y_a) 
#         sigma_a = np.std(diff_y_a) 
#         angle_p0 = [1, mean_a, sigma_a]
#         return distance_p0, angle_p0
    
    
    
    
# distance_p0, angle_p0 = th_m.measurement_model()
# print("distance measurement model: {}\n".format(distance_p0))
# print("angle measurement model: {}".format(angle_p0))

